In [7]:
import numpy as np
import cv2 as cv
import glob

# print("exit")
# exit()
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
gray = []
images = glob.glob('./doc/img/*.jpg')
print(images)
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

img = cv.imread('./doc/img/left12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('./doc/img/calibresult.png', dst)

# undistort
mapx, mapy = cv.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('./doc/img/calibresult.png', dst)

mean_error = 0
for i in range(len(objpoints)):#xrange(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )



np.savez("./doc/img/left_intrinsics",
    ret, mtx, dist, rvecs, tvecs,
    newcameramtx, roi)

['./doc/img\\left.jpg', './doc/img\\left01.jpg', './doc/img\\left02.jpg', './doc/img\\left03.jpg', './doc/img\\left04.jpg', './doc/img\\left05.jpg', './doc/img\\left06.jpg', './doc/img\\left07.jpg', './doc/img\\left08.jpg', './doc/img\\left09.jpg', './doc/img\\left11.jpg', './doc/img\\left12.jpg', './doc/img\\left13.jpg', './doc/img\\left14.jpg']
total error: 0.023686000375385676


In [18]:
import numpy as np
import cv2 as cv
import glob
# Load previously saved data
# with np.load('./doc/img/B.npz') as X:
#     mtx, dist, _, _ = [X[i] for i in ('mtx','dist','rvecs','tvecs')

file = "./doc/img/left_intrinsics.npz"
load = np.load(file)
# print(dir(load),load.files)

ret = load['arr_0']
mtx = load['arr_1']
dist = load['arr_2']
rvecs = load['arr_3']
tvecs = load['arr_4']
newcameramtx = load['arr_5']
roi = load['arr_6']
print("loaded file from",file,'\n', ret,'\n',newcameramtx,roi)

def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

## 
def drawCube(img, corners, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    # draw ground floor in green
    img = cv.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)
    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    # draw top layer in red color
    img = cv.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])
##


criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

for fname in glob.glob('./doc/img/left*.jpg'):
    img = cv.imread(fname)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    ret, corners = cv.findChessboardCorners(gray, (7,6),None)
    if ret == True:
        corners2 = cv.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        # np.savetxt("./doc/img/left_PnP.txt",objp, corners2, mtx, dist)
        # print("*****",objp, corners2, mtx, dist)
        print("*****",objp.shape, corners2.shape, mtx.shape, dist.shape)
        ret,rvecs, tvecs = cv.solvePnP(objp, corners2, mtx, dist)
        # project 3D points to image plane
        imgpts, jac = cv.projectPoints(axis, rvecs, tvecs, mtx, dist)
        img = draw(img,corners2,imgpts)
        cv.imshow('img',img)
        k = cv.waitKey(0) & 0xFF
        if k == ord('s'):
            cv.imwrite("./doc/img/"+fname[:6]+'.png', img)
        break
cv.destroyAllWindows()

loaded file from ./doc/img/left_intrinsics.npz 
 0.155369069100788 
 [[458.64099121   0.         343.09157581]
 [  0.         457.19467163 233.83376475]
 [  0.           0.           1.        ]] [ 11  22 615 438]


NameError: name 'self' is not defined

In [12]:
np.__version__
# ?np.load

'1.18.1'